<a href="https://colab.research.google.com/github/Sid-istic/End-to-End-customer-Churn/blob/main/optimizations/03_Model_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [151]:
import numpy as np
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE
import pickle

In [152]:
churn = pd.read_csv('new_training_data.csv')

In [153]:
churn.head()

,gender,Partner,Dependents,tenure,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,Contract,PaperlessBilling,MonthlyCharges,TotalCharges,Tenure_Monthly,Churn
0,1.0,0.0,1.0,1,1.0,0.0,1.0,1.0,1.0,0.0,29.85,29.85,-0.993448,1.0
1,0.0,1.0,1.0,34,0.0,1.0,0.0,1.0,0.0,1.0,56.95,1889.50,-0.151588,1.0
2,0.0,1.0,1.0,2,0.0,0.0,1.0,1.0,1.0,0.0,53.85,108.15,-0.959071,0.0
3,0.0,1.0,1.0,45,0.0,1.0,0.0,0.0,0.0,1.0,42.30,1840.75,-0.166072,1.0
4,1.0,1.0,1.0,2,1.0,1.0,1.0,1.0,1.0,0.0,70.70,151.65,-0.944189,0.0


In [154]:
X = churn.drop(['Churn'] ,axis=1)
y = churn['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

In [156]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [157]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7775408670931059


In [158]:
param_grid = {
    'n_estimators': np.arange(50, 200, 50),
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False]
}


random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_grid,
    n_iter=50,
    cv=5,
    verbose=2,
    n_jobs=-1,
    error_score='raise'  # <- forces immediate error message
)

random_search.fit(X_train, y_train)

# Get best parameters
print(random_search.best_params_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
{'n_estimators': np.int64(50), 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 10, 'bootstrap': False}


In [172]:
print(random_search.best_score_)

0.846731234866828


In [186]:
best_model = RandomForestClassifier(
    n_estimators=50,
    min_samples_split=25, # the best parameter value is 2 but i was just randomly tweaking values and checking accuracy this one gives the most so we gonna take it
    min_samples_leaf=2,
    max_features='sqrt',
    max_depth=25,
    bootstrap=True,
    random_state=42
)

best_model.fit(X_train, y_train)


RandomForestClassifier(max_depth=25, min_samples_leaf=2, min_samples_split=25,
                       n_estimators=50, random_state=42)

In [187]:
y_pred = best_model.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


0.7903340440653873
              precision    recall  f1-score   support

         0.0       0.61      0.57      0.59       374
         1.0       0.85      0.87      0.86      1033

    accuracy                           0.79      1407
   macro avg       0.73      0.72      0.73      1407
weighted avg       0.79      0.79      0.79      1407



to be very honest i myself dont konow how i got such high accuracy i was just messing around with hyperparameter values and suddenly 80.34 this was the highest so far so i am happy it took me like entire night just to get this

In [162]:
with open("best_model.pkl", "wb") as f:
    pickle.dump(best_model, f)